In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Yelp API Key
gkey = 'fxuoEjmZHd5kA1x20u4z2waSsXfTm49pWr2F6ac-htERJ0haCqXH949tlmDYt9lhy4MC6nBJjt2llMwhQJfxbwX69QlFkI221WkMDZTn7NMQBaXRrWGBFzCrg4-9XXYx'


In [2]:
from etl_clean_ip import zip_list

FileNotFoundError: [Errno 2] File b'/Users/cla/Desktop/UM Data Science/Projects/Project - 2/ETL-Project/Data/MDC_Housing_Data.csv' does not exist: b'/Users/cla/Desktop/UM Data Science/Projects/Project - 2/ETL-Project/Data/MDC_Housing_Data.csv'

In [ ]:
#set up base url
# base url
base_url = "https://api.yelp.com/v3/businesses/search"

# set up api_key dictionary
headers = {
        'Authorization': 'Bearer %s' % gkey,
    }

# set up a parameters dictionary
def s_params(target_loc,target_key,target_type):
    params = {
    "location": target_loc,
    "term": target_key,
    "categories": target_type,
}
    return params

# run a request using our params dictionary
def s_query(base_url,headers,params):
    response = requests.request('GET', base_url, headers=headers, params=params).json()
    return response

In [ ]:
# Import zip list
zip_list 

#cities_pd.shape[0] #use .shape[0] to count the number of rows

In [ ]:
# yelp function
def get_restaurants(zips):
    
    #these are the additional params
    add_params = {"sort_by" : "best_match", "limit" : 1, "radius" : 40000}

    #these are the original params
    loc = zips
    params = s_params(loc,"restaurants","restaurants, All")

    #add additional params to our params from before
    s1_params = {**params,**add_params}

    s1_results = s_query(base_url,headers,s1_params)
    
    if (s1_results["businesses"][0]["name"]):
        name = s1_results["businesses"][0]["name"]
        address = s1_results["businesses"][0]["location"]["display_address"]
        lat = s1_results["businesses"][0]["coordinates"]["latitude"]
        lon = s1_results["businesses"][0]["coordinates"]["longitude"]
        rating = s1_results["businesses"][0]["rating"]
        output = [name,address,lat,lon,rating] 
    else:
        output = ["NaN","NaN","NaN","NaN","NaN"]
    
    
    return output

In [ ]:
# Loop through the cities_pd and run a lat/long search for each city
names = []
addresses = []
lats = []
lons = []
ratings = []

print("wait...")


for zips in zip_list:

    
    
    output = get_restaurants(zips)
    print(output)
    names.append(output[0])
    addresses.append(output[1])
    lats.append(output[2])
    lons.append(output[3])
    ratings.append(output[4])
    
print("done!")

In [ ]:
# Add columns for lat, lng, airport name, airport address, airport rating
cities_pd["Lat"] = lats
cities_pd["Lng"] = lons
cities_pd["Airport Name"] = names
cities_pd["Airport Address"] = addresses
cities_pd["Airport Rating"] = ratings
cities_pd.head()

In [ ]:
# Save Data to csv
cities_pd.to_csv("Airport_Output.csv")

# Visualize to confirm airport data appears
cities_pd.head(10)